In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import os
import nltk
from nltk.translate.bleu_score import corpus_bleu
import zipfile
import shutil


In [5]:
import zipfile
import os
import shutil

def extract_images_with_captions(zip_path, image_names_path, captions_path, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    images_output_dir = os.path.join(output_dir, "images")
    os.makedirs(images_output_dir, exist_ok=True)
    captions_output_file = os.path.join(output_dir, "captions.txt")
    
    # Read required image names
    with open(image_names_path, 'r') as f:
        image_names = {line.strip() for line in f}
    
    # Read captions
    captions = {}
    with open(captions_path, 'r') as f:
        for line in f:
            parts = line.strip().split("\t", 1)  # Split only on the first tab
            if len(parts) == 2:
                base_name, index = parts[0].rsplit("#", 1)  # Extract image name and index
                if base_name in image_names:
                    if base_name not in captions:
                        captions[base_name] = []
                    captions[base_name].append((index, parts[1]))  # Store with index
    
    # Extract images
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            image_name = os.path.basename(file)
            if image_name in image_names:
                zip_ref.extract(file, images_output_dir)
    
    # Write all captions to a single file
    with open(captions_output_file, 'w') as f:
        for image_name in image_names:
            if image_name in captions:
                for index, caption in sorted(captions[image_name], key=lambda x: int(x[0])):
                    f.write(f"{image_name}#{index}\t{caption}\n")
    
    print("Extraction complete! Images saved and captions stored in a single file.")



In [6]:
# Example usage
extract_images_with_captions(
    zip_path="Image captioning dataset-20250330T101538Z-001/Image captioning dataset/Images.zip",
    image_names_path="Image captioning dataset-20250330T101538Z-001/Image captioning dataset/image_mapping and captions/Team28/image_names.txt",
    captions_path="Image captioning dataset-20250330T101538Z-001/Image captioning dataset/image_mapping and captions/Team28/captions.txt",
    output_dir="output"
)


Extraction complete! Images saved and captions stored in a single file.
